In [83]:
import requests
import scrapy

import json
import re

In [84]:
base_url = 'http://goheels.com'
url = base_url + '/roster.aspx?path=mbball'
headers = {'User-Agent': 'UNC Journo Class'}
resp = requests.get(url, headers=headers)


In [85]:
body_bytes = resp.content

body_str = body_bytes.decode('utf-8')

In [86]:
sel = scrapy.Selector(text=body_str)

In [87]:
#selects the first table using [index]
#table = sel.css('.sidearm-table')[0]
table = sel.css('table')[0]
table

<Selector xpath='descendant-or-self::table' data='<table class="sidearm-table sidearm-tabl'>

In [88]:
col_names = table.css('th').xpath('string()').extract()
#col_names = table.css('tr:nth-child(1) th').xpath('string()').extract()

In [89]:
col_names

['No.', 'Name', 'Pos.', 'Ht.', 'Wt.', 'Yr.', 'Hometown / High School']

In [90]:
#other_rows = table.css('tr:not(:nth-child(1))')
other_rows = table.css('tr')[1:]

In [91]:
other_rows

[<Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n                                  '>,
 <Selector xpath='descendant-or-self::tr' data='<tr>\r\n    

In [92]:
players = []
for r in other_rows:
    data = {}
    for i, d in enumerate(r.css('td')):
        a = d.css('a')
        if a:
            t = a.xpath('text()').extract()[0]
            data['href'] = a.xpath('@href').extract()[0]
        else:
            t = d.xpath('text()').extract()[0]
        data[col_names[i]] = t
    players.append(data)

In [93]:
players

[{'Hometown / High School': 'Columbia, S.C. / Hammond School',
  'Ht.': '6-1',
  'Name': 'Seventh Woods',
  'No.': '0',
  'Pos.': 'G',
  'Wt.': '185',
  'Yr.': 'So.',
  'href': '/roster.aspx?rp_id=13521'},
 {'Hometown / High School': 'Greensboro, N.C. / Wesleyan Christian Academy',
  'Ht.': '6-6',
  'Name': 'Theo Pinson',
  'No.': '1',
  'Pos.': 'F/G',
  'Wt.': '220',
  'Yr.': 'Sr.',
  'href': '/roster.aspx?rp_id=13515'},
 {'Hometown / High School': 'Apopka, Fla. / Lake Highland Preparatory',
  'Ht.': '6-0',
  'Name': 'Joel Berry II',
  'No.': '2',
  'Pos.': 'G',
  'Wt.': '195',
  'Yr.': 'Sr.',
  'href': '/roster.aspx?rp_id=13508'},
 {'Hometown / High School': 'Guilderland, N.Y. / Northfield Mount Hermon School (Mass.)',
  'Ht.': '6-3',
  'Name': 'Andrew Platek',
  'No.': '3',
  'Pos.': 'G',
  'Wt.': '195',
  'Yr.': 'Fr.',
  'href': '/roster.aspx?rp_id=13528'},
 {'Hometown / High School': 'Douglasville, Ga. / Douglas County',
  'Ht.': '6-5',
  'Name': 'Brandon Robinson',
  'No.': '4',


In [94]:
# my starting point for just getting the information from the rows
by_player_num = {}
for row in other_rows:
    player_num = row.css('td')[0].xpath('string()').extract()[0]
    player_name = row.css('td')[1].xpath('string()').extract()[0]
    player_position = row.css('td')[2].xpath('string()').extract()[0]
    player_num_data = []
    player_name_data = []
    
    by_player_num[player_num] = player_name, player_position 


In [95]:
by_player_num

{'0': ('Seventh Woods', 'G'),
 '1': ('Theo Pinson', 'F/G'),
 '11': ('Shea Rush', 'F'),
 '13': ('Cameron Johnson', 'G'),
 '14': ('Kane Ma', 'G'),
 '15': ('Garrison Brooks', 'F'),
 '2': ('Joel Berry II', 'G'),
 '21': ('Sterling Manley', 'F'),
 '22': ('Walker Miller', 'F'),
 '24': ('Kenny Williams', 'G'),
 '25': ('Aaron Rohlman', 'F'),
 '3': ('Andrew Platek', 'G'),
 '30': ('K.J. Smith', 'G'),
 '32': ('Luke Maye', 'F'),
 '4': ('Brandon Robinson', 'G'),
 '42': ('Brandon Huffman', 'F')}

In [96]:
#populate all the data to the object organized by number
by_player_num = {}
player_data = []
for row in other_rows:
    
    player_num = row.css('td')[0].xpath('string()').extract()[0]
    player_name = row.css('td')[1].xpath('string()').extract()[0]
    # get the second cell, then the link, then just the href
    link = row.css('td')[1]
    link = link.css('a')
    player_href= link.xpath('@href').extract()
    player_position = row.css('td')[2].xpath('string()').extract()[0]
    player_num_data = []
    player_data = {
        "name": player_name,
        "position": player_position,
        "href": player_href[0],
    }

    by_player_num[player_num] = player_data 
   

In [97]:
by_player_num

{'0': {'href': '/roster.aspx?rp_id=13521',
  'name': 'Seventh Woods',
  'position': 'G'},
 '1': {'href': '/roster.aspx?rp_id=13515',
  'name': 'Theo Pinson',
  'position': 'F/G'},
 '11': {'href': '/roster.aspx?rp_id=13518',
  'name': 'Shea Rush',
  'position': 'F'},
 '13': {'href': '/roster.aspx?rp_id=13525',
  'name': 'Cameron Johnson',
  'position': 'G'},
 '14': {'href': '/roster.aspx?rp_id=14170',
  'name': 'Kane Ma',
  'position': 'G'},
 '15': {'href': '/roster.aspx?rp_id=13522',
  'name': 'Garrison Brooks',
  'position': 'F'},
 '2': {'href': '/roster.aspx?rp_id=13508',
  'name': 'Joel Berry II',
  'position': 'G'},
 '21': {'href': '/roster.aspx?rp_id=13526',
  'name': 'Sterling Manley',
  'position': 'F'},
 '22': {'href': '/roster.aspx?rp_id=13527',
  'name': 'Walker Miller',
  'position': 'F'},
 '24': {'href': '/roster.aspx?rp_id=13520',
  'name': 'Kenny Williams',
  'position': 'G'},
 '25': {'href': '/roster.aspx?rp_id=13517',
  'name': 'Aaron Rohlman',
  'position': 'F'},
 '3':

In [98]:
# now get data from secondary, profile page
for player in list(by_player_num.values()):
    new_url = 'http://goheels.com' + player["href"]
    resp = requests.get(new_url, headers={'User-Agent': 'UNC Journo Class'},)
    content = resp.content.decode('utf-8')
    sel = scrapy.Selector(text=content)
    #selects the image url
    player_image = sel.css('.sidearm-roster-player-image img').xpath("@src").extract()[0]
    # selects the biography
    player_bio = sel.css('#sidearm-roster-player-bio').xpath('string()').extract()[0]
    #player_data = by_player_num[player_num]

    player["image"] = player_image
    #player["bio"] = player_bio
        
    
    #by_player_num[player_num] = player_data
    


In [99]:
def fetch_bio(player):
    player_url = base_url + player['href']
    print('Fetch bio', player_url)
    resp = requests.get(player_url, headers=headers)
    player_txt = resp.content.decode('utf-8')
    sel = scrapy.Selector(text=player_txt)
    player['sel'] = sel
    player['bio'] = sel.css('#sidearm-roster-player-bio').xpath('string()').extract()[0]
    player['img'] = sel.css('.sidearm-roster-player-image img').xpath('@src').extract()[0]

In [100]:
js_obj_rx = re.compile(r'.*?responsive-roster-bio\.ashx.*?(?P<obj>{.*?})')

In [101]:
def fetch_stats(player):
    text = player['sel'].xpath('string()').extract()[0]
    parts = text.split('$.getJSON("/services/')[1:]
    captured = js_obj_rx.findall(''.join(parts))
    clean_objs = []
    for obj_str in captured:
        # We only want the stats object...
        if 'stats' not in obj_str:
            continue

        obj_str = obj_str.replace('{', '').replace('}', '')
        obj_str = obj_str.replace("'", '').replace('"', '')
        obj_pairs = obj_str.split(',')
        obj_pairs = [x.split(":") for x in obj_pairs]
        clean_pairs = []
        for pair in obj_pairs:
            clean_pairs.append(['"{}"'.format(p.strip()) for p in pair])
        colonized = [":".join(p) for p in clean_pairs]
        commas = ','.join(colonized)
        json_str = "{" + commas + "}"
        clean_objs.append(json.loads(json_str))
    
    player['stats_url'] = stats_url = (
        "http://goheels.com/services/responsive-roster-bio.ashx?"
        "type={type}&rp_id={rp_id}&path={path}&year={year}"
        "&player_id={player_id}"
    ).format(**clean_objs[0])
    
    print('Fetch stats', stats_url)
    resp = requests.get(stats_url, headers=headers)
    json_stats = json.loads(resp.content.decode("utf-8"))
    player['raw_stats'] = json_stats

In [102]:
for p in players:
    fetch_bio(p)
    fetch_stats(p)

Fetch bio http://goheels.com/roster.aspx?rp_id=13521
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=13521&path=mbball&year=2017&player_id=4736
Fetch bio http://goheels.com/roster.aspx?rp_id=13515
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=13515&path=mbball&year=2017&player_id=4636
Fetch bio http://goheels.com/roster.aspx?rp_id=13508
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=13508&path=mbball&year=2017&player_id=4632
Fetch bio http://goheels.com/roster.aspx?rp_id=13528
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=13528&path=mbball&year=2017&player_id=5171
Fetch bio http://goheels.com/roster.aspx?rp_id=13516
Fetch stats http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=13516&path=mbball&year=2017&player_id=4733
Fetch bio http://goheels.com/roster.aspx?rp_id=13518
Fetch stats http://goheels.com/services/respons

In [103]:
players[0]

{'Hometown / High School': 'Columbia, S.C. / Hammond School',
 'Ht.': '6-1',
 'Name': 'Seventh Woods',
 'No.': '0',
 'Pos.': 'G',
 'Wt.': '185',
 'Yr.': 'So.',
 'bio': '\r\n                        Biography\r\n                                                    \r\n                            FRESHMAN SEASON (2016-17)\r\n\r\nTied a school record by appearing in all 40 games • Averaged 7.7 minutes per game in the primary relief role behind Joel Berry II at the point • Had 49 assists, 42 turnovers and 21 steals • Had 25 assists and 13 turnovers in ACC regular-season play • Made two steals five times • Was on the floor for five minutes vs. Louisville as UNC out-scored the Cards, 10-2 • Played his bestgame in conference play at Duke when he scored four points and had a team-high four assists in eight minutes • Played a seven and a half minute stretch in the first half as UNC cut Duke’s lead from three to one • Was the first time he led UNC in assists since Long Beach State on 11/15 • Had t

In [104]:
p = [p for p in players if p['Name'] == 'Theo Pinson'][0]

In [105]:
txt = p['raw_stats']['career_stats']

In [106]:
sel = scrapy.Selector(text=txt)

In [107]:
sel.css('section')

[]

In [108]:
def parse_stats(player):
    stats = {}
    for raw_key, raw_val in player['raw_stats'].items():
        txt = player['raw_stats'][raw_key]
        if not txt:
            print('Skipping {} for {}'.format(raw_key, player['Full Name']))
            continue
        sel = scrapy.Selector(text=txt)
        # Get all the tables
        for section in sel.css('section'):
            title = section.css('h5').xpath('string()').extract()[0]
            cols = section.css('tr')[0].css('th').xpath('string()').extract()
            print('NEW SECTION', title)
            print('COLS', cols)
            these_stats = []
            print('TRS', section.css('tr'))
            for r in section.css('tr')[1:]:
                print('row', r.xpath('string()').extract()[0].replace('\r', '').replace('\n', '').strip())
                s = {}
                for i, d in enumerate(r.css('td'), 1):
                    s[cols[i].lower()] = d.xpath('string()').extract()[0]
                yr = r.css('th').xpath('string()')
                if yr:
                    yr = yr.extract()[0]
                    if yr.lower() in ('total', 'season'):
                        print('SKIPPING...')
                        continue
                    print('THE YR IS', yr)
                    s['year'] = yr
                these_stats.append(s)
                print('THE STATS ARE', these_stats)
            existing = stats.get(raw_key, {})
            existing[title] = these_stats
            stats[raw_key] = existing
    player['stats'] = stats

In [109]:
p = [p for p in players if p['Name'] == 'Theo Pinson'][0]
parse_stats(p)

NEW SECTION Game-By-Game Statistics
COLS ['Date', 'Opponent', 'GS', 'MIN', 'FGM/A', '%', '3FG/A', '%', 'FTM/A', '%', 'OFF', 'DEF', 'TOT', 'AVG', 'PF', 'AST', 'T/O', 'BLK', 'STL', 'PTS', 'AVG']
TRS [<Selector xpath='descendant-or-self::tr' data='<tr>\r\n                        <th scope='>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>       

IndexError: list index out of range

In [110]:
for p in players:
    parse_stats(p)

NEW SECTION Game-By-Game Statistics
COLS ['Date', 'Opponent', 'GS', 'MIN', 'FGM/A', '%', '3FG/A', '%', 'FTM/A', '%', 'OFF', 'DEF', 'TOT', 'AVG', 'PF', 'AST', 'T/O', 'BLK', 'STL', 'PTS', 'AVG']
TRS [<Selector xpath='descendant-or-self::tr' data='<tr>\r\n                        <th scope='>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>                        \r\n          '>, <Selector xpath='descendant-or-self::tr' data='<tr>       

IndexError: list index out of range

In [115]:
[p for p in players if p['Name'] == 'Theo Pinson'][0]['stats'] #mysterious, but the dump works

KeyError: 'stats'

In [116]:
to_dump = [p.copy() for p in players]
for p in to_dump:
    p.pop('sel')
    for k in list(p.keys()):
        if 'raw' in k:
            p.pop(k)
with open('scraped_players.json', 'w') as f:
    json.dump(to_dump, f)

In [117]:
cat scraped_players.json | cut -c 1-100

[{"No.": "0", "href": "/roster.aspx?rp_id=13521", "Name": "Seventh Woods", "Pos.": "G", "Ht.": "6-1"


In [118]:
to_dump[0]

{'Hometown / High School': 'Columbia, S.C. / Hammond School',
 'Ht.': '6-1',
 'Name': 'Seventh Woods',
 'No.': '0',
 'Pos.': 'G',
 'Wt.': '185',
 'Yr.': 'So.',
 'bio': '\r\n                        Biography\r\n                                                    \r\n                            FRESHMAN SEASON (2016-17)\r\n\r\nTied a school record by appearing in all 40 games • Averaged 7.7 minutes per game in the primary relief role behind Joel Berry II at the point • Had 49 assists, 42 turnovers and 21 steals • Had 25 assists and 13 turnovers in ACC regular-season play • Made two steals five times • Was on the floor for five minutes vs. Louisville as UNC out-scored the Cards, 10-2 • Played his bestgame in conference play at Duke when he scored four points and had a team-high four assists in eight minutes • Played a seven and a half minute stretch in the first half as UNC cut Duke’s lead from three to one • Was the first time he led UNC in assists since Long Beach State on 11/15 • Had t